We can get the data for game franchises from Giantbomb. My api key is f2577eac7f52b72170fc1e8c5f0cebb778f38155

In [55]:
import requests
import json
import pandas as pd
import re
from pathlib import Path
import time
import os

## Scraping franchise IDs

Create the directory to store the raw info

In [56]:
franchises_dir = Path('raw_json\\game_franchises\\franchises')
franchises_dir.mkdir(parents=True, exist_ok=True)

Get all the data from the franchise ids and save as json files

In [ ]:
user_agent = "Joshdharris"
headers = {'User-Agent': user_agent}

for x in range(0, 5550, 100):
    filename = "page{}-{}.json".format(x, x+99)
    if not Path(franchises_dir, filename).is_file():
        url = "https://www.giantbomb.com/api/franchises/?api_key=f2577eac7f52b72170fc1e8c5f0cebb778f38155&format=json&field_list=guid,name,description&offset={}".format(x)
        response = json.loads((requests.get(url, headers=headers)).content)
        with open(Path(franchises_dir, filename), "w") as fileOut:
            json.dump(response, fileOut)

This function will be used multiple times for the raw json files. Here we create an empty dataframe, then open all the json files in a specified directory and add them to the dataframe, before returning the dataframe

In [57]:
def create_dataframe(desired_name, raw_directory, add_path):
    #Create empty dataframe
    desired_name = pd.DataFrame(data=[], columns=None)

    #This line creates a list of all the json files in the directory
    files = list(Path(raw_directory).glob('*.json'))
    
    #Iterate through all json files in directory and add to dataframe
    for file in files:
        with open(file) as filein:
            tempDF = pd.read_json(filein)
            if add_path == True:
                tempDF["path"] = str(file)
            desired_name = pd.concat([desired_name, tempDF])  
    return desired_name

Collate all the downloaded files into a single dataframe

In [58]:
franchise_ids = create_dataframe("franchise_ids", franchises_dir, False)
franchise_ids.sample()

,error,limit,offset,number_of_page_results,number_of_total_results,status_code,results,version
70,OK,100,500,100,5524,1,{'description': '<p>The National Hockey League...,1


In [59]:
franchise_ids.shape

(5524, 8)

Remove unnecessary rows. Again, create a function as it's used multiple times

In [60]:
def drop_unnecessary_columns(dataframe):
    dataframe = dataframe.drop(["error","limit", "offset", "number_of_page_results", "number_of_total_results", "status_code", "version"], axis = 1)
    return dataframe

In [61]:
franchise_ids = drop_unnecessary_columns(franchise_ids)
franchise_ids.shape

(5524, 1)

Extract information from results

In [62]:
franchise_ids["guid"] = [d.get('guid') for d in franchise_ids.results]
franchise_ids["name"] = [d.get('name') for d in franchise_ids.results]
franchise_ids["description"] = [d.get('description') for d in franchise_ids.results]

Get list of all franchise ids.

In [63]:
franchise_id_list = franchise_ids["guid"].tolist()
print(len(franchise_id_list))

5524


Make sure we have no duplicates of the ids

In [64]:
franchise_ids.guid.duplicated().sum()

0

Check out the resultant dataframe

In [65]:
franchise_ids.sample()

,results,guid,name,description
44,"{'description': None, 'guid': '3025-977', 'nam...",3025-977,LEGO Racers,None


Drop the results. Write to csv file

In [67]:
franchise_ids = franchise_ids.drop("results", axis = 1)
franchise_ids.sample()

In [70]:
franchise_ids.to_csv("clean_franchise_ids.csv")

## Scraping game IDs

Create directory for game franchises

In [71]:
games_dir = Path('raw_json\\game_franchises\\games')
games_dir.mkdir(parents=True, exist_ok=True)

Download json files for each franchise. We have a cap of 200 requests an hour, hence we're using a time restriction within the loop to prevent the api from blocking us.

In [ ]:
for id in franchise_id_list:
    filename = "{}.json".format(id)
    if not Path(games_dir, filename).is_file():
        url = "https://www.giantbomb.com/api/franchise/{}/?api_key=f2577eac7f52b72170fc1e8c5f0cebb778f38155&format=json&field_list=games".format(id)
        response = json.loads((requests.get(url, headers=headers)).content)
        with open(Path(games_dir, filename), "w") as fileOut:
            json.dump(response, fileOut)
        time.sleep(18)

Create a games dataframe from all the games in the directory

In [72]:
games_ids = create_dataframe("games_ids", games_dir, True)
games_ids.sample()

,error,limit,offset,number_of_page_results,number_of_total_results,status_code,results,version,path
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,raw_json\game_franchises\games\3025-4488.json


Edit the path to be franchise ids and rename

In [73]:
games_ids["path"] = games_ids["path"].apply(lambda x : x[31:-5])
games_ids = games_ids.rename(columns = {"path":"franchise_id"})
games_ids

,error,limit,offset,number_of_page_results,number_of_total_results,status_code,results,version,franchise_id
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-1
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-10
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-100
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-101
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-102
...,...,...,...,...,...,...,...,...,...
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-94
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-95
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-97
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-98


### Retrieving missing values

For some reason we have only 5346 results, but 5524 franchise ids. This is, we discovered, because the api is a little buggy. For some reason, having the field-list restriction on returns an empty result for the missing apis, so the workaround is to download all the information and extract the game information from that. 

In [75]:
# Lists all json files in directory
games_in_dir = os.listdir(games_dir)
# Removes .json extension from names
for i in range(len(games_in_dir)):
    games_in_dir[i] = games_in_dir[i][:-5]
# Creates a new list of all the games in the franchise list, but not in the directory
missing_apis = [x for x in franchise_id_list if x not in games_in_dir]
# Check number of missing files
print(len(missing_apis))

178


We'll put the missing files into a separate directory and make a separate dataframe so we don't mess up anything with the current dataframe

In [76]:
missing_games_dir = Path('raw_json\\game_franchises\\games\\missing')
missing_games_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
for id in missing_apis:
    filename = "{}.json".format(id)
    if not Path(missing_games_dir, filename).is_file():
        url = "https://www.giantbomb.com/api/franchise/{}/?api_key=f2577eac7f52b72170fc1e8c5f0cebb778f38155&format=json".format(id)
        response = json.loads((requests.get(url, headers=headers)).content)
        with open(Path(missing_games_dir, filename), "w") as fileOut:
            json.dump(response, fileOut)

In [77]:
missing_games_ids = create_dataframe("missing_games_ids", missing_games_dir, True)
missing_games_ids

,error,limit,offset,number_of_page_results,number_of_total_results,status_code,results,version,path
aliases,OK,1,0,1,1,1,None,1,raw_json\game_franchises\games\missing\3025-10...
api_detail_url,OK,1,0,1,1,1,https://www.giantbomb.com/api/franchise/3025-1...,1,raw_json\game_franchises\games\missing\3025-10...
characters,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,raw_json\game_franchises\games\missing\3025-10...
concepts,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,raw_json\game_franchises\games\missing\3025-10...
date_added,OK,1,0,1,1,1,2008-08-25 13:52:36,1,raw_json\game_franchises\games\missing\3025-10...
...,...,...,...,...,...,...,...,...,...
locations,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,raw_json\game_franchises\games\missing\3025-99...
name,OK,1,0,1,1,1,Shaman King,1,raw_json\game_franchises\games\missing\3025-99...
objects,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,raw_json\game_franchises\games\missing\3025-99...
people,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,raw_json\game_franchises\games\missing\3025-99...


We'll remove any rows that don't contain the game listings, rename the path column and extract the games ids

In [78]:
missing_games_ids = missing_games_ids.drop(missing_games_ids.index.difference(['games']))
missing_games_ids["path"] = missing_games_ids["path"].apply(lambda x : x[39:-5])
missing_games_ids = missing_games_ids.rename(columns = {"path":"franchise_id"})

<ipython-input-78-7780a6cf1ad9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_games_ids["path"] = missing_games_ids["path"].apply(lambda x : x[39:-5])


In [79]:
missing_games_ids.shape

(178, 9)

In [80]:
missing_games_ids.sample()

,error,limit,offset,number_of_page_results,number_of_total_results,status_code,results,version,franchise_id
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-921


Finally, we'll combine our two dataframes to reintegrate the missing values into our original dataframe

In [81]:
games_ids = pd.concat([games_ids, missing_games_ids])
games_ids.franchise_id.count()

5524

And we can now see we have all the values we expected.

### Working with the combined dataframe

Remove the unnecessary columns, so we're jsut left with results and franchise ids.

In [82]:
games_ids = drop_unnecessary_columns(games_ids)
games_ids.sample()

,results,franchise_id
games,[],3025-3734


Explode the results so every game has its own row

In [83]:
games_ids = games_ids.explode("results")
games_ids.count()

results         35117
franchise_id    35139
dtype: int64

We have some empty results so we'll drop those before moving on

In [85]:
games_ids = games_ids.dropna()
games_ids.count()

results         35117
franchise_id    35117
dtype: int64

We extract individual game names and ids. We can also extract the individual game ids so we can do further analysis on other games

In [86]:
games_ids[["game_id", "name", "api_url"]] = games_ids["results"].apply(lambda x: pd.Series([x["id"], x["name"], x["api_detail_url"]]))

In [87]:
games_ids["game_id"].nunique()

29208

There's 29208 unique game ids which implies that around 6000 of our games are in multiple franchises. we can look at the ids of the games in the most franchises

In [88]:
games_ids["game_id"].value_counts().head()

6949     7
33847    6
73766    6
569      6
38379    5
Name: game_id, dtype: int64

In [89]:
games_ids = games_ids.drop("results", axis = 1)
games_ids.sample()

,franchise_id,game_id,name,api_url
games,3025-468,36303,One Piece: Treasure Wars 2 - Welcome to Buggyland,https://www.giantbomb.com/api/game/3030-36303/


## Scraping additional game metadata

We need additional metadata for the games so we'll create the relevant folder to hold the raw json, extract all the unique ids, combined with the api address. Then download it all

In [4]:
game_dir = Path('raw_json\\game_franchises\\game')
game_dir.mkdir(parents=True, exist_ok=True)

In [47]:
game_api_dict = dict(zip(games_ids["game_id"].unique(), games_ids["api_url"].unique()))
print(len(game_api_dict))

29208


In [ ]:
for game_id, api_url in game_api_dict.items():
    filename = "{}.json".format(game_id)
    if not Path(game_dir, filename).is_file():
        url = "{}?api_key=f2577eac7f52b72170fc1e8c5f0cebb778f38155&format=json".format(api_url)
        response = json.loads((requests.get(url, headers=headers)).content)
        with open(Path(game_dir, filename), "w") as fileOut:
            json.dump(response, fileOut)
        time.sleep(18)

### Combining the game metadata

We want to also combine the year for each game in the giantbomb database to the relevant game. We create a list for each game and their release year then combine them into a dataframe

In [5]:
metadataframe = create_dataframe("metadataframe", game_dir, True)
metadataframe.sample()

KeyboardInterrupt: 

In [ ]:
metadataframe["path"] = metadataframe["path"].apply(lambda x : x[29:-5])
metadataframe = metadataframe.rename(columns = {"path":"game_id"})

In [14]:
with open ("raw_json/game_franchises/game/1.json") as file:
    temp = pd.read_json(file)
temp["results"]

aliases                                                    Desert Strike Advance
api_detail_url                        https://www.giantbomb.com/api/game/3030-1/
characters                                                                  None
concepts                       [{'api_detail_url': 'https://www.giantbomb.com...
date_added                                                   2008-04-01 01:32:48
date_last_updated                                            2021-08-15 16:32:37
deck                           A top-down isometric helicopter shoot 'em up o...
description                    <h2>Overview</h2><p>Desert Strike: Return to t...
developers                     [{'api_detail_url': 'https://www.giantbomb.com...
expected_release_day                                                        None
expected_release_month                                                      None
expected_release_quarter                                                    None
expected_release_year       

In [15]:
ids = []
release_years = []
aliases = []
developers =[]
genres =[]
platforms =[]
publishers =[]
rating = []


files = list(Path("raw_json/game_franchises/game/").glob('*.json'))
for file in files:
    with open(file) as filein:
        tempDF = pd.read_json(filein)
        ids.append(tempDF["results"].get("id"))
        release_years.append(tempDF["results"].get("expected_release_year"))
        aliases.append(tempDF["results"].get("aliases"))
        developers.append(tempDF["results"].get("developers"))
        genres.append(tempDF["results"].get("genres")) 
        platforms.append(tempDF["results"].get("platforms"))
        publishers.append(tempDF["results"].get("publishers")) 
        rating.append(tempDF["results"].get("original_game_rating")) 
        print(file)

raw_json\game_franchises\game\1.json
raw_json\game_franchises\game\1000.json
raw_json\game_franchises\game\10007.json
raw_json\game_franchises\game\10009.json
raw_json\game_franchises\game\1001.json
raw_json\game_franchises\game\10012.json
raw_json\game_franchises\game\10013.json
raw_json\game_franchises\game\10014.json
raw_json\game_franchises\game\10015.json
raw_json\game_franchises\game\1002.json
raw_json\game_franchises\game\10020.json
raw_json\game_franchises\game\10021.json
raw_json\game_franchises\game\10022.json
raw_json\game_franchises\game\10024.json
raw_json\game_franchises\game\1003.json
raw_json\game_franchises\game\10030.json
raw_json\game_franchises\game\10031.json
raw_json\game_franchises\game\10032.json
raw_json\game_franchises\game\10034.json
raw_json\game_franchises\game\10036.json
raw_json\game_franchises\game\10040.json
raw_json\game_franchises\game\10041.json
raw_json\game_franchises\game\10043.json
raw_json\game_franchises\game\10045.json
raw_json\game_franchises

raw_json\game_franchises\game\10406.json
raw_json\game_franchises\game\10408.json
raw_json\game_franchises\game\1041.json
raw_json\game_franchises\game\10410.json
raw_json\game_franchises\game\10416.json
raw_json\game_franchises\game\10419.json
raw_json\game_franchises\game\1042.json
raw_json\game_franchises\game\10425.json
raw_json\game_franchises\game\10426.json
raw_json\game_franchises\game\10429.json
raw_json\game_franchises\game\10431.json
raw_json\game_franchises\game\10434.json
raw_json\game_franchises\game\10438.json
raw_json\game_franchises\game\1044.json
raw_json\game_franchises\game\10442.json
raw_json\game_franchises\game\10447.json
raw_json\game_franchises\game\10448.json
raw_json\game_franchises\game\1045.json
raw_json\game_franchises\game\10450.json
raw_json\game_franchises\game\10459.json
raw_json\game_franchises\game\10460.json
raw_json\game_franchises\game\10461.json
raw_json\game_franchises\game\10465.json
raw_json\game_franchises\game\10466.json
raw_json\game_franch

raw_json\game_franchises\game\10834.json
raw_json\game_franchises\game\10837.json
raw_json\game_franchises\game\10839.json
raw_json\game_franchises\game\1084.json
raw_json\game_franchises\game\10842.json
raw_json\game_franchises\game\10845.json
raw_json\game_franchises\game\10846.json
raw_json\game_franchises\game\10847.json
raw_json\game_franchises\game\10850.json
raw_json\game_franchises\game\10857.json
raw_json\game_franchises\game\10858.json
raw_json\game_franchises\game\10861.json
raw_json\game_franchises\game\10863.json
raw_json\game_franchises\game\10865.json
raw_json\game_franchises\game\10866.json
raw_json\game_franchises\game\10867.json
raw_json\game_franchises\game\10868.json
raw_json\game_franchises\game\1087.json
raw_json\game_franchises\game\10871.json
raw_json\game_franchises\game\10873.json
raw_json\game_franchises\game\10874.json
raw_json\game_franchises\game\10876.json
raw_json\game_franchises\game\10877.json
raw_json\game_franchises\game\10878.json
raw_json\game_fran

raw_json\game_franchises\game\11289.json
raw_json\game_franchises\game\1129.json
raw_json\game_franchises\game\11290.json
raw_json\game_franchises\game\11292.json
raw_json\game_franchises\game\11293.json
raw_json\game_franchises\game\11297.json
raw_json\game_franchises\game\113.json
raw_json\game_franchises\game\11300.json
raw_json\game_franchises\game\11302.json
raw_json\game_franchises\game\11305.json
raw_json\game_franchises\game\11310.json
raw_json\game_franchises\game\11311.json
raw_json\game_franchises\game\11312.json
raw_json\game_franchises\game\11314.json
raw_json\game_franchises\game\11315.json
raw_json\game_franchises\game\11321.json
raw_json\game_franchises\game\11322.json
raw_json\game_franchises\game\11324.json
raw_json\game_franchises\game\11325.json
raw_json\game_franchises\game\11328.json
raw_json\game_franchises\game\11329.json
raw_json\game_franchises\game\11331.json
raw_json\game_franchises\game\11332.json
raw_json\game_franchises\game\11339.json
raw_json\game_franc

raw_json\game_franchises\game\11793.json
raw_json\game_franchises\game\11794.json
raw_json\game_franchises\game\11796.json
raw_json\game_franchises\game\11799.json
raw_json\game_franchises\game\1180.json
raw_json\game_franchises\game\11800.json
raw_json\game_franchises\game\11802.json
raw_json\game_franchises\game\1181.json
raw_json\game_franchises\game\11811.json
raw_json\game_franchises\game\11812.json
raw_json\game_franchises\game\11813.json
raw_json\game_franchises\game\11816.json
raw_json\game_franchises\game\11817.json
raw_json\game_franchises\game\11820.json
raw_json\game_franchises\game\11822.json
raw_json\game_franchises\game\11823.json
raw_json\game_franchises\game\11826.json
raw_json\game_franchises\game\11829.json
raw_json\game_franchises\game\11830.json
raw_json\game_franchises\game\11831.json
raw_json\game_franchises\game\11832.json
raw_json\game_franchises\game\11837.json
raw_json\game_franchises\game\11841.json
raw_json\game_franchises\game\11843.json
raw_json\game_fran

...

raw_json\game_franchises\game\87300.json
raw_json\game_franchises\game\87301.json
raw_json\game_franchises\game\87302.json
raw_json\game_franchises\game\87303.json
raw_json\game_franchises\game\87304.json
raw_json\game_franchises\game\87305.json
raw_json\game_franchises\game\87306.json
raw_json\game_franchises\game\87307.json
raw_json\game_franchises\game\87308.json
raw_json\game_franchises\game\87313.json
raw_json\game_franchises\game\87314.json
raw_json\game_franchises\game\87317.json
raw_json\game_franchises\game\8732.json
raw_json\game_franchises\game\87321.json
raw_json\game_franchises\game\87322.json
raw_json\game_franchises\game\87323.json
raw_json\game_franchises\game\87324.json
raw_json\game_franchises\game\87325.json
raw_json\game_franchises\game\87326.json
raw_json\game_franchises\game\87327.json
raw_json\game_franchises\game\87329.json
raw_json\game_franchises\game\8733.json
raw_json\game_franchises\game\87330.json
raw_json\game_franchises\game\87331.json
raw_json\game_fran

raw_json\game_franchises\game\88014.json
raw_json\game_franchises\game\88015.json
raw_json\game_franchises\game\88020.json
raw_json\game_franchises\game\88028.json
raw_json\game_franchises\game\8804.json
raw_json\game_franchises\game\88043.json
raw_json\game_franchises\game\88044.json
raw_json\game_franchises\game\88046.json
raw_json\game_franchises\game\8805.json
raw_json\game_franchises\game\88056.json
raw_json\game_franchises\game\88057.json
raw_json\game_franchises\game\88058.json
raw_json\game_franchises\game\88059.json
raw_json\game_franchises\game\8806.json
raw_json\game_franchises\game\88061.json
raw_json\game_franchises\game\88063.json
raw_json\game_franchises\game\88064.json
raw_json\game_franchises\game\88066.json
raw_json\game_franchises\game\88068.json
raw_json\game_franchises\game\88069.json
raw_json\game_franchises\game\8807.json
raw_json\game_franchises\game\88071.json
raw_json\game_franchises\game\88072.json
raw_json\game_franchises\game\88075.json
raw_json\game_franch

raw_json\game_franchises\game\8865.json
raw_json\game_franchises\game\88664.json
raw_json\game_franchises\game\88666.json
raw_json\game_franchises\game\8867.json
raw_json\game_franchises\game\88673.json
raw_json\game_franchises\game\88675.json
raw_json\game_franchises\game\88677.json
raw_json\game_franchises\game\88679.json
raw_json\game_franchises\game\88681.json
raw_json\game_franchises\game\88683.json
raw_json\game_franchises\game\88687.json
raw_json\game_franchises\game\88688.json
raw_json\game_franchises\game\8869.json
raw_json\game_franchises\game\88692.json
raw_json\game_franchises\game\88694.json
raw_json\game_franchises\game\88695.json
raw_json\game_franchises\game\8870.json
raw_json\game_franchises\game\88702.json
raw_json\game_franchises\game\88703.json
raw_json\game_franchises\game\88714.json
raw_json\game_franchises\game\88715.json
raw_json\game_franchises\game\88725.json
raw_json\game_franchises\game\88729.json
raw_json\game_franchises\game\8873.json
raw_json\game_franchi

raw_json\game_franchises\game\9256.json
raw_json\game_franchises\game\9258.json
raw_json\game_franchises\game\9259.json
raw_json\game_franchises\game\9264.json
raw_json\game_franchises\game\9265.json
raw_json\game_franchises\game\9267.json
raw_json\game_franchises\game\9269.json
raw_json\game_franchises\game\927.json
raw_json\game_franchises\game\9275.json
raw_json\game_franchises\game\9276.json
raw_json\game_franchises\game\9278.json
raw_json\game_franchises\game\928.json
raw_json\game_franchises\game\9281.json
raw_json\game_franchises\game\9282.json
raw_json\game_franchises\game\9283.json
raw_json\game_franchises\game\9286.json
raw_json\game_franchises\game\9287.json
raw_json\game_franchises\game\9288.json
raw_json\game_franchises\game\9289.json
raw_json\game_franchises\game\9292.json
raw_json\game_franchises\game\9295.json
raw_json\game_franchises\game\9299.json
raw_json\game_franchises\game\93.json
raw_json\game_franchises\game\9301.json
raw_json\game_franchises\game\9303.json
raw_

raw_json\game_franchises\game\9707.json
raw_json\game_franchises\game\9710.json
raw_json\game_franchises\game\9714.json
raw_json\game_franchises\game\9716.json
raw_json\game_franchises\game\9719.json
raw_json\game_franchises\game\9720.json
raw_json\game_franchises\game\9722.json
raw_json\game_franchises\game\9725.json
raw_json\game_franchises\game\9726.json
raw_json\game_franchises\game\9727.json
raw_json\game_franchises\game\9730.json
raw_json\game_franchises\game\9734.json
raw_json\game_franchises\game\9737.json
raw_json\game_franchises\game\9741.json
raw_json\game_franchises\game\9742.json
raw_json\game_franchises\game\9743.json
raw_json\game_franchises\game\9744.json
raw_json\game_franchises\game\9745.json
raw_json\game_franchises\game\9747.json
raw_json\game_franchises\game\9752.json
raw_json\game_franchises\game\9753.json
raw_json\game_franchises\game\9755.json
raw_json\game_franchises\game\9757.json
raw_json\game_franchises\game\9759.json
raw_json\game_franchises\game\9761.json


In [17]:
id_years_dict = list(zip(ids, release_years, aliases, developers, genres, platforms, publishers, rating))
id_year_df = pd.DataFrame.from_dict(id_years_dict)
id_year_df = id_year_df.rename(columns = {0:"id", 1:"release_year", 2:"aliases", 3: "developers", 4:"genres", 5: "platforms", 6: "publishers", 7: "rating"})
id_year_df = id_year_df.sort_values("id")
id_year_df

,id,release_year,2,3,4,5,6,7
0,1,1992.0,Desert Strike Advance,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...
9677,3,NaN,None,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,None
14192,4,1986.0,None,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,None
21240,6,NaN,SVR 2007,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...
26238,8,NaN,Battle Formula,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,None
...,...,...,...,...,...,...,...,...
28707,88822,NaN,None,None,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...
28708,88824,NaN,None,None,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...
28709,88831,NaN,None,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,None
28710,88834,NaN,None,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,None


In [18]:
id_year_df = id_year_df.rename(columns = {2:"aliases", 3: "developers", 4:"genres", 5: "platforms", 6: "publishers", 7: "rating"})
id_year_df

,id,release_year,aliases,developers,genres,platforms,publishers,rating
0,1,1992.0,Desert Strike Advance,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...
9677,3,NaN,None,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,None
14192,4,1986.0,None,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,None
21240,6,NaN,SVR 2007,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...
26238,8,NaN,Battle Formula,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,None
...,...,...,...,...,...,...,...,...
28707,88822,NaN,None,None,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...
28708,88824,NaN,None,None,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...
28709,88831,NaN,None,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,None
28710,88834,NaN,None,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,None


In [43]:
id_year_df["developers"] = id_year_df["developers"].apply(lambda x: [d["name"] for d in x] if x is not None else None)
id_year_df["genres"] = id_year_df["genres"].apply(lambda x: [d["name"] for d in x] if x is not None else None)
id_year_df["platforms"] = id_year_df["platforms"].apply(lambda x: [d["name"] for d in x] if x is not None else None)
id_year_df["publishers"] = id_year_df["publishers"].apply(lambda x: [d["name"] for d in x] if x is not None else None)
id_year_df["rating"] = id_year_df["rating"].apply(lambda x: [d["name"] for d in x] if x is not None else None)

In [52]:
id_year_df

,id,release_year,aliases,developers,genres,platforms,publishers,rating
0,1,1992.0,Desert Strike Advance,"[Electronic Arts, Visual Concepts, Budcat Crea...","[Action, Shoot 'Em Up]","[Amiga, Game Boy, Game Boy Advance, Game Gear,...","[Electronic Arts, Domark Software, Malibu Game...","[ESRB: K-A, ESRB: E]"
9677,3,NaN,None,[Kernel Kaput],"[Action, Block-Breaking]",[PC],"[Alawar Entertainment, Inc.]",None
14192,4,1986.0,None,"[The Software Toolworks, Software Country, Gam...","[Strategy, Trivia/Board Game]","[Amiga, Amstrad CPC, Apple II, Atari ST, Commo...","[The Software Toolworks, Software Country, Ele...",None
21240,6,NaN,SVR 2007,"[Yuke's Co. Ltd., Digital Hearts Co., Ltd. ]","[Action, Sports, Wrestling]","[PlayStation Portable, PlayStation 2, Xbox 360]",[THQ],"[ESRB: T, PEGI: 16+, CERO: C]"
26238,8,NaN,Battle Formula,[Tokai Engineering],[Vehicular Combat],[Nintendo Entertainment System],[Sunsoft],None
...,...,...,...,...,...,...,...,...
28707,88822,NaN,None,None,[Driving/Racing],"[PC, PlayStation 4, Nintendo Switch, PlayStati...",[Koei Tecmo],[CERO: A]
28708,88824,NaN,None,None,[Simulation],[Game Boy Advance],[TDK Core],[CERO: All Ages]
28709,88831,NaN,None,[Magical Company],[Trivia/Board Game],[PlayStation],[Magical Company],None
28710,88834,NaN,None,[Sega],[Adventure],"[Mac, PC]",[Sega],None


In [54]:
id_year_df.to_csv("clean_game_metadata.csv")

Combine the dataframes and remove the redundant columns

In [90]:
games_ids = pd.merge(games_ids, id_year_df, how='left', left_on='game_id', right_on='id')
games_ids = games_ids.drop(["id","api_url"], axis = 1)
games_ids

,franchise_id,game_id,name,release_year,aliases,developers,genres,platforms,publishers,rating
0,3025-1,544,Super Mario All-Stars & Super Mario World,1994.0,Super Mario All-Stars and Super Mario World,"[Nintendo EAD, SRD Co. Ltd.]","[Compilation, Platformer]",[Super Nintendo Entertainment System],[Nintendo],[ESRB: K-A]
1,3025-1,6649,Super Mario Sunshine,NaN,None,[Nintendo EAD],"[Platformer, Action-Adventure]","[GameCube, Nintendo Switch]",[Nintendo],"[ESRB: E, OFLC: G, PEGI: 3+]"
2,3025-1,7314,Super Mario Bros. Deluxe,NaN,Super Mario Bros. DX,[Nintendo R&D2],"[Action, Adventure, Compilation, Puzzle, Platf...","[Game Boy Color, Nintendo 3DS eShop]",[Nintendo],[ESRB: E]
3,3025-1,7358,Super Mario RPG: Legend of the Seven Stars,NaN,Mario RPG\r\nSMRPG,"[Squaresoft, Square Enix]","[Adventure, Role-Playing]","[Super Nintendo Entertainment System, Wii Shop...",[Nintendo],"[ESRB: K-A, PEGI: 3+, ESRB: E, OFLC: G, CERO: A]"
4,3025-1,7406,Super Mario Bros. 2,NaN,Super Mario USA\nSuper Mario Bros. USA\nSMB2,"[Nintendo EAD, SRD Co. Ltd.]",[Platformer],"[Nintendo Entertainment System, Wii Shop, Nint...",[Nintendo],"[ESRB: E, CERO: All Ages]"
...,...,...,...,...,...,...,...,...,...,...
35112,3025-999,26577,Shaman King: Funbari Spirits,NaN,None,[Dimps Corporation],[Fighting],[PlayStation 2],"[Bandai Co., Ltd.]",None
35113,3025-999,26575,"Shaman King: Legacy of the Spirits, Soaring Hawk",NaN,None,None,[Role-Playing],[Game Boy Advance],[Konami],None
35114,3025-999,26581,Shaman King Chou Senjiryokketsu: Meramera/Funb...,NaN,None,[Studio Saizensen],[Action],[Game Boy Color],[King Records],None
35115,3025-999,26580,Shaman King: Soul Fight!,NaN,None,"[Bandai Co., Ltd.]",[Fighting],[GameCube],"[Bandai Co., Ltd.]",None


Finally we'll export all the collected data to a csv file. 

In [91]:
games_ids.to_csv("clean_giantbomb_games_db.csv")